# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# Описание данных

* `id` — уникальный идентификатор скважины


* `f0`, ` f1`, `f2` — три признака точек (неважно, что они означают, но сами признаки значимы)


* `product` — объём запасов в скважине (тыс. баррелей)

**Условия задачи:**

* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).

* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.

* Бюджет на разработку скважин в регионе — 10 млрд рублей.

* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Импорт-используемых-библиотек" data-toc-modified-id="Импорт-используемых-библиотек-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт используемых библиотек</a></span></li><li><span><a href="#Чтение-файлов-с-данными" data-toc-modified-id="Чтение-файлов-с-данными-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Чтение файлов с данными</a></span></li><li><span><a href="#Изучение-общей-информации" data-toc-modified-id="Изучение-общей-информации-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Изучение общей информации</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели</a></span><ul class="toc-item"><li><span><a href="#Данные-геологоразведки-первого-региона" data-toc-modified-id="Данные-геологоразведки-первого-региона-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Данные геологоразведки первого региона</a></span></li><li><span><a href="#Данные-геологоразведки-второго-региона" data-toc-modified-id="Данные-геологоразведки-второго-региона-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Данные геологоразведки второго региона</a></span></li><li><span><a href="#Данные-геологоразведки-третьего-региона" data-toc-modified-id="Данные-геологоразведки-третьего-региона-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Данные геологоразведки третьего региона</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

## Загрузка и подготовка данных

### Импорт используемых библиотек

In [48]:
# импорт библиотеки pandas
import pandas as pd

# импорт библиотеки numpy
import numpy as np

# импорт библиотеки matplotlib
import matplotlib.pyplot as plt

# импорт функции train_test_split из библиотеки sklearn
from sklearn.model_selection import train_test_split

# импорт алгоритма линейной регрессии из библиотеки sklearn
from sklearn.linear_model import LinearRegression

# импорт кросс-валидации из библиотеки sklearn
from sklearn.model_selection import cross_val_score

# импорт коэффициента детерминации из библиотеки sklearn
from sklearn.metrics import r2_score

# импорт средней квадратичной ошибки из библиотеки sklearn
from sklearn.metrics import mean_squared_error

# импорт библиотеки stats из библиотеки scipy
from scipy import stats as st

# импорт генератора случайных чисел из библиотеки numpy
from numpy.random import RandomState

### Чтение файлов с данными

In [49]:
# чтение файлов с данными
data_1 = pd.read_csv('/datasets/geo_data_0.csv')
data_2 = pd.read_csv('/datasets/geo_data_1.csv')
data_3 = pd.read_csv('/datasets/geo_data_2.csv')

### Изучение общей информации

In [50]:
# получение первых 5 строк датасетов
print('\033[1m' + 'Первые 5 строк датасетов'.center(44) + '\033[0m')
print('=' * 44)
print('\033[1m\033[35m' + 'Данные геологоразведки первого региона'.center(44) + '\033[0m\033[0m')
display(data_1.head())
print()
print('\033[1m\033[31m' + 'Данные геологоразведки второго региона'.center(44) + '\033[0m\033[0m')
display(data_2.head())
print()
print('\033[1m\033[32m' + 'Данные геологоразведки третьего региона'.center(44) + '\033[0m\033[0m')
display(data_3.head())

          Первые 5 строк датасетов          
   Данные геологоразведки первого региона   


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647



   Данные геологоразведки второго региона   


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305



  Данные геологоразведки третьего региона   


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [51]:
# получение общей информации о данных датасетов
print('\033[1m' + 'Общая информация о данных'.center(38) + '\033[0m')
print('=' * 38)
print('\033[1m\033[35m' + 'Данные геологоразведки первого региона'.center(38) + '\033[0m\033[0m')
data_1.info()
print()
print('\033[1m\033[31m' + 'Данные геологоразведки второго региона'.center(38) + '\033[0m\033[0m')
data_2.info()
print()
print('\033[1m\033[32m' + 'Данные геологоразведки третьего региона'.center(38) + '\033[0m\033[0m')
data_3.info()

      Общая информация о данных       
Данные геологоразведки первого региона
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Данные геологоразведки второго региона
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Данные геолог

In [52]:
# проверка на наличие дубликатов
print('\033[1m' + 'Количество дубликатов'.center(40) + '\033[0m')
print('=' * 40)
print('\033[1m\033[35m' + 'Данные геологоразведки первого региона' + '\033[0m\033[0m')
print(data_1.duplicated().sum())
print()
print('\033[1m\033[31m' + 'Данные геологоразведки второго региона' + '\033[0m\033[0m')
print(data_2.duplicated().sum())
print()
print('\033[1m\033[32m' + 'Данные геологоразведки третьего региона' + '\033[0m\033[0m')
print(data_3.duplicated().sum())

         Количество дубликатов          
Данные геологоразведки первого региона
0

Данные геологоразведки второго региона
0

Данные геологоразведки третьего региона
0


Все датасеты имеют правильное обозначение столбцов, не имеют пропусков и дубликатов в данных и имеют подходящий тип данных. Удалим столбец с данными об уникальном идентификаторе скважины, чтобы он не мешал обучению модели.

In [53]:
# удаление столбца из датасетов
data_1.drop('id', axis=1, inplace=True)
data_2.drop('id', axis=1, inplace=True)
data_3.drop('id', axis=1, inplace=True)

In [54]:
# получение описательной статистики данных датасетов
print('\033[1m' + 'Описательная статистика'.center(56) + '\033[0m')
print('=' * 56)
print('\033[1m\033[35m' + 'Данные геологоразведки первого региона'.center(56) + '\033[0m\033[0m')
display(data_1.describe())
print()
print('\033[1m\033[31m' + 'Данные геологоразведки второго региона'.center(56) + '\033[0m\033[0m')
display(data_2.describe())
print()
print('\033[1m\033[32m' + 'Данные геологоразведки третьего региона'.center(56) + '\033[0m\033[0m')
display(data_3.describe())

                Описательная статистика                 
         Данные геологоразведки первого региона         


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347



         Данные геологоразведки второго региона         


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408



        Данные геологоразведки третьего региона         


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Во всех регионах присутствуют скважины с нулевым объемом запасов, но так как при разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки, то оставим их и не будем удалять из датасетов.

### Выводы

Данные готовы к обучению модели. В них нет ни пропусков, ни дубликатов. Масштабировать их также не требуется.

## Обучение и проверка модели

### Данные геологоразведки первого региона

In [55]:
# разделите данные на обучающую и валидационную выборки
data_1_train, data_1_valid = train_test_split(data_1, test_size=0.25, random_state=12345)

In [56]:
# проверка размера выборок
data_1_train.shape[0], data_1_valid.shape[0]

(75000, 25000)

In [57]:
# создание переменные для признаков и целевого признака
features_train_1 = data_1_train.drop(['product'], axis=1)
target_train_1 = data_1_train['product']
features_valid_1 = data_1_valid.drop(['product'], axis=1)
target_valid_1 = data_1_valid['product']

In [58]:
# проверка размера выборок признаков
features_train_1.shape, features_valid_1.shape

((75000, 3), (25000, 3))

In [59]:
# обучение алгоритма линейной регрессии
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predictions_valid_1 = model_1.predict(features_valid_1)

In [60]:
# вычисление rmse
rmse_1 = mean_squared_error(target_valid_1, predictions_valid_1)**0.5
print('RMSE:', rmse_1)

# вычисление коэффициента детерминации
r2_1 = r2_score(target_valid_1, predictions_valid_1)
print('R2:', r2_1)

# вычисление среднего запаса сырья
print('Средний запас сырья:', predictions_valid_1.mean())

RMSE: 37.5794217150813
R2: 0.27994321524487786
Средний запас сырья: 92.59256778438035


### Данные геологоразведки второго региона

In [61]:
# разделите данные на обучающую и валидационную выборки
data_2_train, data_2_valid = train_test_split(data_2, test_size=0.25, random_state=12345)

In [62]:
# проверка размера выборок
data_2_train.shape[0], data_2_valid.shape[0]

(75000, 25000)

In [63]:
# создание переменные для признаков и целевого признака
features_train_2 = data_2_train.drop(['product'], axis=1)
target_train_2 = data_2_train['product']
features_valid_2 = data_2_valid.drop(['product'], axis=1)
target_valid_2 = data_2_valid['product']

In [64]:
# проверка размера выборок признаков
features_train_2.shape, features_valid_2.shape

((75000, 3), (25000, 3))

In [65]:
# обучение алгоритма линейной регрессии
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predictions_valid_2 = model_2.predict(features_valid_2)

In [66]:
# вычисление rmse
rmse_2 = mean_squared_error(target_valid_2, predictions_valid_2)**0.5
print('RMSE:', rmse_2)

# вычисление коэффициента детерминации
r2_2 = r2_score(target_valid_2, predictions_valid_2)
print('R2:', r2_2)

# вычисление среднего запаса сырья
print('Средний запас сырья:', predictions_valid_2.mean())

RMSE: 0.893099286775617
R2: 0.9996233978805127
Средний запас сырья: 68.728546895446


### Данные геологоразведки третьего региона

In [67]:
# разделите данные на обучающую и валидационную выборки
data_3_train, data_3_valid = train_test_split(data_3, test_size=0.25, random_state=12345)

In [68]:
# проверка размера выборок
data_3_train.shape[0], data_3_valid.shape[0]

(75000, 25000)

In [69]:
# создание переменные для признаков и целевого признака
features_train_3 = data_3_train.drop(['product'], axis=1)
target_train_3 = data_3_train['product']
features_valid_3 = data_3_valid.drop(['product'], axis=1)
target_valid_3 = data_3_valid['product']

In [70]:
# проверка размера выборок признаков
features_train_3.shape, features_valid_3.shape

((75000, 3), (25000, 3))

In [71]:
# обучение алгоритма линейной регрессии
model_3 = LinearRegression()
model_3.fit(features_train_3, target_train_3)
predictions_valid_3 = model_3.predict(features_valid_3)

In [72]:
# вычисление rmse
rmse_3 = mean_squared_error(target_valid_3, predictions_valid_3)**0.5
print('RMSE:', rmse_3)

# вычисление коэффициента детерминации
r2_3 = r2_score(target_valid_3, predictions_valid_3)
print('R2:', r2_3)

# вычисление среднего запаса сырья
print('Средний запас сырья:', predictions_valid_3.mean())

RMSE: 40.02970873393434
R2: 0.20524758386040443
Средний запас сырья: 94.96504596800489


### Выводы

In [73]:
metrics = [['RMSE', rmse_1, rmse_2, rmse_3], 
            ['R2', r2_1, r2_2, r2_3], 
            ['Средний запас сырья:', 
            predictions_valid_1.mean(), predictions_valid_2.mean(), predictions_valid_3.mean()]]
columns_name = ['Параметры', 'Первый регион', 'Второй регион', 'Третий регион']
table_metrics = pd.DataFrame(data = metrics, columns = columns_name)
table_metrics

,Параметры,Первый регион,Второй регион,Третий регион
0,RMSE,37.579422,0.893099,40.029709
1,R2,0.279943,0.999623,0.205248
2,Средний запас сырья:,92.592568,68.728547,94.965046


Из полученной таблицы видно, что модель второго региона имеет самую низкую ошибку и почти идеальный коэффициент детерминации. Метрики моделей первого и третьего региона похожи.
Запасы сырья в первом и третьем регионе практически равны, а во втором их количество меньше.

## Подготовка к расчёту прибыли

In [74]:
# создание констант
SCOUTING_POINTS = 500
BEST_POINTS = 200
BUDGET = 10**10
PRICE_PER_BARREL = 450
REVENUE_PER_UNIT = PRICE_PER_BARREL * 1000
PROBABILITY_OF_LOSSES = 0.025

In [75]:
# расчет объема сырья для безубыточной разработки
breakeven_point = BUDGET / REVENUE_PER_UNIT / BEST_POINTS
print('Достаточный объём сырья для безубыточной разработки новой скважины:', breakeven_point)

Достаточный объём сырья для безубыточной разработки новой скважины: 111.11111111111111


In [76]:
mean_product = {'Средний запас сырья': [predictions_valid_1.mean(), predictions_valid_2.mean(), predictions_valid_3.mean()]}
table_mean = pd.DataFrame(data=mean_product,
                     index= ['Первый регион', 'Второй регион', 'Третий регион'])
table_mean

,Средний запас сырья
Первый регион,92.592568
Второй регион,68.728547
Третий регион,94.965046


Из данных видно, что согласно средним значениям объема сырья не хватает для того чтобы выйти на точку безубыточности.

In [77]:
# функция для расчета прибыли
def profit(target, predictions, count):
    predictions_sorted = predictions.sort_values(ascending=False)
    selected = target[predictions_sorted.index][:count]
    return (REVENUE_PER_UNIT * selected.sum()) - BUDGET

## Расчёт прибыли и рисков 

In [78]:
# добавим случайности нашей функции, чтобы каждый раз брался разный сэмпл
state = RandomState(12345) 

In [79]:
def bootstrap(target, predictions):
    values = []
    target = target.reset_index(drop=True)
    predictions = pd.Series(predictions)
    
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state = state)
        predictions_subsample = predictions[target_subsample.index]
        values.append(profit(target_subsample, predictions_subsample, BEST_POINTS))
    
    # находим доверительный интервал и риск убытков
    values = pd.Series(values)
    lower = values.quantile(.025)
    upper = values.quantile(.975)
    risk = len(values[values<0])/len(values)
    
    # <выводим результат>
    print('Средняя прибыль:','{:.2f}'.format(values.mean()))
    print('Начало доверительного интервала:','{:.2f}'.format(lower))
    print('Конец доверительного интервала:','{:.2f}'.format(upper))
    print('Риск убытков:','{:.2f}'.format(risk))
    
    return values

In [80]:
print('\033[1m\033[35m' + 'Данные геологоразведки первого региона'.center(45) + '\033[0m\033[0m')
values = bootstrap(target_valid_1, predictions_valid_1)

    Данные геологоразведки первого региона   
Средняя прибыль: 425938526.91
Начало доверительного интервала: -102090094.84
Конец доверительного интервала: 947976353.36
Риск убытков: 0.06


In [81]:
print('\033[1m\033[35m' + 'Данные геологоразведки второго региона'.center(45) + '\033[0m\033[0m')
values = bootstrap(target_valid_2, predictions_valid_2)

    Данные геологоразведки второго региона   
Средняя прибыль: 518259493.70
Начало доверительного интервала: 128123231.43
Конец доверительного интервала: 953612982.07
Риск убытков: 0.00


In [82]:
print('\033[1m\033[35m' + 'Данные геологоразведки третьего региона'.center(45) + '\033[0m\033[0m')
values = bootstrap(target_valid_3, predictions_valid_3)

   Данные геологоразведки третьего региона   
Средняя прибыль: 420194005.34
Начало доверительного интервала: -115852609.16
Конец доверительного интервала: 989629939.84
Риск убытков: 0.06


## Выводы

Проведя анализ между тремя регионами можно сделать вывод, что второй регион самый удачный для разработки скважин, т.к. наша модель показала самую лучшую точность в данном регионе, что понижает ошибку выбора скважин по сравнению с другими регионами, а так же данный регион имеет самую высокую среднюю прибыль по региону, высокое значение верхней границы доверительного интервала и положительное значение нижнего доверительного интервала, а также нулеврй риск.